In [2]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from function_util import *
from IMP import IMP
from SMP1 import SMP1
from SMP2 import SMP2
from FIS1 import FIS1
from FIS2 import FIS2
from FIS3 import FIS3

In [17]:
# load all the data 
gdf_utm = pd.read_pickle('gdf_utm.pkl')
edges_utm = pd.read_pickle('edges_utm.pkl')
nodes_utm = pd.read_pickle('nodes_utm.pkl')

# select error bound 
err_size = 38
# select cut off point for smp1, for use 60 following Quddus paper
cutoff = 60 
# select number of no match needed to break iteration 
no_match_max = 3

# intialize no match counter 
no_match = 0 
# initialize matched trajectory list
matched_point_list = []
# initialize matched edge list 
matched_edge_list = []
# initialize SMP1 result list 
res_list =[]
# initialize counter for matched_list 
count = 0

point_index = 0

# Run IMP Algorithm to find first point 
point_index, curr_edge = IMP(point_index, gdf_utm, edges_utm, 7)

#store current edge 
matched_edge_list.append(curr_edge)

# matched current location with current edge 
curr_loc = gdf_utm.iloc[[point_index]]
matched_point = point_matching(curr_loc, curr_edge.iloc[0])
# find longitude and latitude for last matched data
matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})

# update previous matched position
last_matched = matched_point

# store matched point 
matched_point_list.append(matched_point)

# move to the next iteration with for loop
start_iter = point_index + 1


for point_index in range(start_iter, len(gdf_utm)):    
    print(point_index)
    # update last location 
    prev_loc = gdf_utm.iloc[[point_index - 1]]

    # update current location 
    curr_loc = gdf_utm.iloc[[point_index]]

    # perform SMP1 
    res = SMP1(curr_loc, curr_edge, prev_loc, last_matched, nodes_utm, edges_utm, gdf_utm)
    # store result list
    res_list.append(res)


    # decide whether vehicle is entering junction or not cut off 
    if res >= cutoff:
        # vehicle is not entering a new edge, matched edge doesnt change
        # update matched edge list 
        matched_edge_list.append(curr_edge)
    elif res < cutoff:
        prev_edge = curr_edge
        # SMP1 detect vehicle is entering new edge, perform SMP 2 to update current edge
        edge_candidate = SMP2(curr_loc, curr_edge, prev_loc, last_matched, err_size, nodes_utm, edges_utm, gdf_utm)

        if edge_candidate is None:
            print(['No edge can be matched at trajectory number', point_index])
            no_match += 1
        else:
            curr_edge = edge_candidate
            # curr_edge['prev_end_node'] = prev_edge.index[0][1]
            # there is no more matched edge, break the loop 
            matched_edge_list.append(curr_edge)
            # update matched point
            matched_point = point_matching(curr_loc, curr_edge.iloc[0])
            # find longitude and latitude for last matched data
            matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
            # store matched point 
            matched_point_list.append(matched_point)
            # update previous matched position
            last_matched = matched_point

    if no_match == no_match_max:
        # stop iteration when there number of no match is exceeded
        break

    # update count 
    count +=  1


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pl

['no edeges intersects with error bound at iteration number', 1]
['no edeges intersects with error bound at iteration number', 2]
['no edeges intersects with error bound at iteration number', 3]
['no edeges intersects with error bound at iteration number', 4]
['no edeges intersects with error bound at iteration number', 5]
['no edeges intersects with error bound at iteration number', 6]
['no edeges intersects with error bound at iteration number', 7]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pl

['no edeges intersects with error bound at iteration number', 8]
['no edeges intersects with error bound at iteration number', 9]
['no edeges intersects with error bound at iteration number', 10]
['no edeges intersects with error bound at iteration number', 11]
['no edeges intersects with error bound at iteration number', 12]
['no edeges intersects with error bound at iteration number', 13]
['edges found at iteration number', 14]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\U

...

C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\U

351
352


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\skfuzzy\membership\generatemf.py:340: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(- c * (x - b)))
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` param

353
354
['No edge can be matched at trajectory number', 354]
355
['No edge can be matched at trajectory number', 355]
356
['No edge can be matched at trajectory number', 356]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
# create dataframe for the results
matched_edge_df = pd.concat(matched_edge_list)
matched_point_df = pd.concat(matched_point_list)

In [12]:
# measure connectivity of the link 
matched_edge_df

osmid  oneway lanes  \
u          v          key                                        
5156897997 9265218120 0               1004161022    True     1   
                      0               1004161022    True     1   
           676106     0               1004161023    True     3   
676106     54235085   0    [16134112, 597880873]    True     1   
5156897997 676106     0               1004161023    True     3   
...                                          ...     ...   ...   
277412620  277412618  0                149958343   False   NaN   
277412618  277412620  0                149958343   False   NaN   
           151003151  0    [24959910, 149958343]   False   NaN   
151003151  277412618  0    [24959910, 149958343]   False   NaN   
                      0    [24959910, 149958343]   False   NaN   

                                                 name        highway maxspeed  \
u          v          key                                                       
5156897997 9265218120 0                 Pischekstraße  tertiary_link      NaN   
                      0                 Pischekstraße  tertiary_link      NaN   
           676106     0                 Pischekstraße      secondary       40   
676106     54235085   0                  Planckstraße      secondary       40   
5156897997 676106     0                 Pischekstraße      secondary       40   
...                                               ...            ...      ...   
277412620  277412618  0                  Diamantenweg    residential       30   
277412618  277412620  0                  Diamantenweg    residential       30   
           151003151  0    [Smaragdweg, Diamantenweg]    residential       30   
151003151  277412618  0    [Smaragdweg, Diamantenweg]    residential       30   
                      0    [Smaragdweg, Diamantenweg]    residential       30   

                          reversed  length     ref  \
u          v          key                            
5156897997 9265218120 0      False  24.387     NaN   
                      0      False  24.387     NaN   
           676106     0      False  27.327  L 1016   
676106     54235085   0      False  76.744  L 1016   
5156897997 676106     0      False  27.327  L 1016   
...                            ...     ...     ...   
277412620  277412618  0      False  35.334     NaN   
277412618  277412620  0       True  35.334     NaN   
           151003151  0      False  70.331     NaN   
151003151  277412618  0       True  70.331     NaN   
                      0       True  70.331     NaN   

                                                                    geometry  \
u          v          key                                                      
5156897997 9265218120 0    LINESTRING (514380.818 5401852.210, 514385.703...   
                      0    LINESTRING (514380.818 5401852.210, 514385.703...   
           676106     0    LINESTRING (514380.818 5401852.210, 514384.965...   
676106     54235085   0    LINESTRING (514399.798 5401871.912, 514411.366...   
5156897997 676106     0    LINESTRING (514380.818 5401852.210, 514384.965...   
...                                                                      ...   
277412620  277412618  0    LINESTRING (512395.329 5403559.981, 512368.847...   
277412618  277412620  0    LINESTRING (512368.847 5403583.444, 512395.329...   
           151003151  0    LINESTRING (512368.847 5403583.444, 512335.411...   
151003151  277412618  0    LINESTRING (512353.652 5403628.876, 512335.411...   
                      0    LINESTRING (512353.652 5403628.876, 512335.411...   

                           ... access  \
u          v          key  ...          
5156897997 9265218120 0    ...    NaN   
                      0    ...    NaN   
           676106     0    ...    NaN   
676106     54235085   0    ...    NaN   
5156897997 676106     0    ...    NaN   
...                        ...    ...   
277412620  277412618  0    ...    NaN   
277412618  277412

In [7]:
point_index

383

# Visualize Result 

In [16]:
%matplotlib tk
# This is how we  visualize edges and error bound 

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = gdf_utm['geometry'].to_frame()
temp = point_locs.plot(ax = ax)

# this code plot all the road system 
edges_utm.plot(ax=ax)

# matched point plot
matched_point_df.plot(ax = ax, color = "Green")

# matched_edge 
matched_edge_df.plot(ax = ax, color = "Black")


<Axes: >